In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
dfd = pd.read_csv('data/train_date.csv.zip', index_col=0, nrows=10000)

In [22]:
cold = dfd.columns.str.extract(r'(L\d+)_(S\d+)_(D\d+)', expand=True)

In [23]:
cold = cold.drop([0], axis=1).drop_duplicates()

In [26]:
cold = [tuple(cold.iloc[i]) for i in cold.index]

In [30]:
cold = pd.MultiIndex.from_tuples(cold, names=['station', 'feature'])

In [32]:
dfd.columns = cold

In [43]:
stations = ['S'+str(x) for x in sorted([int(i[1:]) for i in cold.levels[0]])]

In [47]:
stations_date = pd.DataFrame({i: dfd[i].apply(min, axis=1) for i in stations})

In [56]:
station_orders = stations_date.apply(lambda x: x.dropna().index.tolist(), axis=1)

In [64]:
station_orders1 = station_orders.apply(lambda x: '_'.join(x))

In [70]:
a = station_orders1.unique()

In [73]:
hash(a[0]) % 2**16

53340

In [74]:
import networkx as nx

In [75]:
G = nx.Graph()

In [77]:
G.add_nodes_from(stations)

In [78]:
G.nodes

NodeView(('S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31', 'S32', 'S33', 'S34', 'S35', 'S36', 'S37', 'S38', 'S39', 'S40', 'S41', 'S42', 'S43', 'S44', 'S45', 'S46', 'S47', 'S48', 'S49', 'S50', 'S51'))

In [79]:
station_orders1

Id
4        S0_S1_S2_S4_S7_S8_S11_S29_S30_S31_S33_S34_S35_S37
6        S12_S13_S14_S16_S18_S20_S21_S29_S30_S33_S34_S3...
7            S0_S1_S2_S5_S6_S8_S10_S29_S30_S33_S34_S35_S37
9            S0_S1_S2_S4_S7_S8_S10_S29_S30_S33_S34_S36_S37
11           S0_S1_S3_S4_S7_S8_S11_S29_S30_S33_S34_S36_S37
13           S0_S1_S3_S4_S7_S8_S10_S29_S30_S33_S34_S36_S37
14       S12_S13_S15_S17_S19_S20_S22_S29_S30_S33_S34_S3...
16                             S26_S29_S30_S33_S34_S35_S37
18           S0_S1_S2_S4_S7_S8_S10_S29_S30_S33_S34_S36_S37
23       S12_S13_S14_S17_S18_S20_S21_S29_S30_S33_S34_S3...
26           S0_S1_S2_S4_S7_S8_S10_S29_S30_S33_S34_S36_S37
27            S0_S1_S3_S5_S6_S8_S9_S29_S30_S33_S34_S36_S37
28            S0_S1_S3_S4_S7_S8_S9_S29_S30_S33_S34_S35_S37
31        S0_S1_S2_S4_S6_S8_S9_S29_S30_S31_S33_S34_S35_S37
34            S0_S1_S2_S5_S6_S8_S9_S29_S30_S33_S34_S36_S37
38            S0_S1_S2_S4_S7_S8_S9_S29_S30_S33_S34_S36_S37
41       S12_S13_S14_S17_S19_S20_S23_S29_S30_S33_S34_

In [122]:
result = pd.read_csv('data/train_numeric.csv.zip', index_col=0, usecols=[0, 969], nrows=10000)

In [126]:
pos_idx = result[result['Response']==1].index
neg_idx = result[result['Response']==0].index

In [134]:
station_orders1.loc[pos_idx] = station_orders1.loc[pos_idx].apply(lambda x: x+'_positive')
station_orders1.loc[neg_idx] = station_orders1.loc[neg_idx].apply(lambda x: x+'_negative')

In [140]:
station_orders1 = pd.DataFrame(station_orders1)

In [151]:
a = station_orders1[0].value_counts()

In [165]:
flows = station_orders1[0].value_counts()

In [167]:
a = flows.index[0]

In [173]:
b = a.split('_')
[(b[i], b[i+1]) for i in range(len(b)-1)]

[('S26', 'S29'),
 ('S29', 'S30'),
 ('S30', 'S33'),
 ('S33', 'S34'),
 ('S34', 'S36'),
 ('S36', 'S37'),
 ('S37', 'negative')]

In [174]:
b

['S26', 'S29', 'S30', 'S33', 'S34', 'S36', 'S37', 'negative']

In [177]:
b = b[:-2]


In [178]:
b

['S26', 'S29', 'S30', 'S33']

In [179]:
G

In [183]:
c = [(b[i], b[i+1]) for i in range(len(b)-1)]

In [184]:
G.add_edges_from(c)

In [202]:
pos=nx.spring_layout(G)
for n, p in pos.items():
    G.node[n]['pos'] = p

In [195]:
import plotly.plotly as py
import plotly.graph_objs as go

In [196]:
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')


In [203]:
for edge in G.edges():
    x0, y0 = G.node[edge[0]]['pos']
    x1, y1 = G.node[edge[1]]['pos']
    edge_trace['x'] += [x0, x1, None]
    edge_trace['y'] += [y0, y1, None]

In [205]:
node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
        # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='YIGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in G.nodes():
    x, y = G.node[node]['pos']
    node_trace['x'].append(x)
    node_trace['y'].append(y)

In [208]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network graph made with Python',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

py.iplot(fig, filename='networkx')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~lygirona/0 or inside your plot.ly account where it is named 'networkx'
